# LangGraph Exercise: Building Stateful LLM Applications with Azure OpenAI

This exercise is designed for experienced technical professionals to gain hands-on experience with **LangGraph**, a library for building robust, stateful, and multi-actor applications with Large Language Models (LLMs). We will use **Azure OpenAI Service** as our LLM provider.

## Goal
Implement two core LangGraph patterns:
1.  **Part A: Conditional Graph (Client Query Routing):** Create a dynamic workflow that routes a user query to one of three specialized agents based on the query's content.
2.  **Part B: Static Graph (Compliance Report Generation):** Build a fixed, sequential workflow for automated document assembly.

## Prerequisites
1.  Python environment with the following packages installed: `langchain`, `langgraph`, `langchain-openai`, `pydantic`.
2.  Access to an Azure OpenAI Service instance with a deployment named `gpt-4.1-mini`.
3.  Your Azure OpenAI API Key and Endpoint.

In [1]:
!pip install langchain langchain-openai langgraph pydantic python-dotenv openai langsmith --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 4.6 MB/s eta 0:00:00


In [2]:
# Configuration and Imports
import os
from typing import Literal, TypedDict

# Core LangChain/LangGraph imports
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# --- Azure OpenAI Configuration ---
# NOTE: Replace these placeholders with your actual Azure OpenAI credentials.
# For a corporate environment, these would typically be loaded from a secure vault or environment variables.
os.environ["LANGSMITH_API_KEY"] = os.environ.get("LANGSMITH_API_KEY", "xxxxxxxxxxxxxx")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get("AZURE_OPENAI_ENDPOINT", "https://eastus.api.cognitive.microsoft.com/")
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_API_KEY", "xxxxxxxxxx")
os.environ["OPENAI_API_VERSION"] = os.environ.get("OPENAI_API_VERSION", "2024-08-01-preview")
os.environ["LANGSMITH_TRACING"] = "true"


AZURE_OPENAI_DEPLOYMENT = "gpt-4.1-mini"


print("Environment variables configured (using placeholders).")
print(f"LLM Deployment: {AZURE_OPENAI_DEPLOYMENT}")

Environment variables configured (using placeholders).
LLM Deployment: gpt-4.1-mini


In [4]:
# Initialize AzureChatOpenAI LLM
try:
    llm = AzureChatOpenAI(
        azure_deployment=AZURE_OPENAI_DEPLOYMENT,
        temperature=0
    )
    print("AzureChatOpenAI LLM initialized successfully.")
except Exception as e:
    print(f"ERROR: Could not initialize AzureChatOpenAI. Please check your API key, endpoint, and deployment name: {e}")
    print("The notebook will continue, but LLM-dependent cells will use placeholders or default to a fallback route.")

AzureChatOpenAI LLM initialized successfully.


## Part A: Client Query Routing Exercise

In this exercise, we simulate a financial services firm where incoming client queries need to be routed to the correct specialist team: **General Inquiry**, **Policy Management**, or **Investment Strategy**. This is a classic use case for a **conditional graph** in LangGraph.

The core component is a **Router Node** that uses the LLM's structured output capabilities to make a routing decision.

In [5]:
# Define State, Pydantic Schema for Routing, and the Router Node Function
# 1. Define the State
# TypedDict is used to define the state for the graph, avoiding full OOP classes.
class RouterState(TypedDict):
    query: str
    route: Literal["general", "policy", "investment"]
    response: str

# 2. Define the Router Schema (Pydantic)
class RouteDecision(BaseModel):
    """The decision on which specialist workflow should handle the client query."""
    route: Literal["general", "policy", "investment"] = Field(
        ...,
        description="The determined route for the query. Must be one of: general, policy, investment."
    )

In [6]:
# 3. Define the Router Prompt and Chain
router_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert financial query router. Analyze the user's query and determine the most appropriate specialist team. Your output MUST be a JSON object conforming to the provided schema."),
    ("human", "Client Query: {query}")
])

In [8]:
# 4. Define the Router Node Function
def route_query(state: RouterState) -> RouterState:
    """Determines the next step based on the query content."""
    print(f"--- Router Node: Analyzing Query: {state['query']} ---")

    try:
        # Bind the Pydantic schema to the LLM for structured output
        router_chain = router_prompt | llm.with_structured_output(RouteDecision)
        decision = router_chain.invoke({"query": state["query"]})
        route = decision.route
        print(f"--- Router Decision: Route to '{route}' ---")
        return {"route": route}
    except Exception as e:
        print(f"ERROR in router chain: {e}. Defaulting to 'general'.")
        # Fallback to general in case of error
        return {"route": "general"}

In [10]:
# Define the three Nodes and the Conditional Edge Function
# Define the three specialist nodes.
# For this exercise, they will simply return a simulated response.

def general_node(state: RouterState) -> RouterState:
    print("--- General Node: Processing Inquiry ---")
    response = f"Response from General Inquiry: Thank you for your general question about our services. We are looking into your request: '{state['query']}'."
    return {"response": response}

def policy_node(state: RouterState) -> RouterState:
    print("--- Policy Node: Processing Inquiry ---")
    response = f"Response from Policy Management: Your query regarding your policy: '{state['query']}' has been forwarded to a Policy Specialist. Expect a detailed response within 24 hours."
    return {"response": response}

def investment_node(state: RouterState) -> RouterState:
    print("--- Investment Node: Processing Inquiry ---")
    response = f"Response from Investment Strategy: We are analyzing your investment-related query: '{state['query']}'. Please note that investment advice is subject to market conditions."
    return {"response": response}

# Define the conditional function for the router
def get_next_step(state: RouterState) -> str:
    """The conditional function that maps the route to the next node name."""
    if state["route"] == "general":
        return "general_node"
    elif state["route"] == "policy":
        return "policy_node"
    elif state["route"] == "investment":
        return "investment_node"
    else:
        # Should not happen if Pydantic is enforced, but good for robustness
        return "general_node"

In [11]:
# Build and Compile the LangGraph for Routing
# 1. Initialize the StateGraph
workflow = StateGraph(RouterState)

# 2. Add the nodes
workflow.add_node("router", route_query)
workflow.add_node("general_node", general_node)
workflow.add_node("policy_node", policy_node)
workflow.add_node("investment_node", investment_node)

# 3. Set the entry point
workflow.set_entry_point("router")

# 4. Add conditional edges from the router
# The conditional edge uses the get_next_step function to decide the next node
workflow.add_conditional_edges(
    "router",
    get_next_step,
    {
        "general_node": "general_node",
        "policy_node": "policy_node",
        "investment_node": "investment_node",
    }
)

# 5. Add finish edges from the agents
workflow.add_edge("general_node", END)
workflow.add_edge("policy_node", END)
workflow.add_edge("investment_node", END)

# 6. Compile the graph
app_router = workflow.compile()

print("LangGraph for Client Query Routing compiled successfully.")

LangGraph for Client Query Routing compiled successfully.


In [12]:
# Test the Routing Graph with three distinct queries
# Test Case 1: General Inquiry
query_1 = "What are your office hours next week?"
print(f" --- Running Test Case 1: {query_1} ---")
result_1 = app_router.invoke({"query": query_1})
print(f"Final State Response: {result_1['response']}")
print(f"Final State Route: {result_1['route']}")

# Test Case 2: Policy Management
query_2 = "I need to update the beneficiary on my term life insurance policy."
print(f" --- Running Test Case 2: {query_2} ---")
result_2 = app_router.invoke({"query": query_2})
print(f"Final State Response: {result_2['response']}")
print(f"Final State Route: {result_2['route']}")

# Test Case 3: Investment Strategy
query_3 = "What is your current outlook on the S&P 500 for the next quarter?"
print(f" --- Running Test Case 3: {query_3} ---")
result_3 = app_router.invoke({"query": query_3})
print(f"Final State Response: {result_3['response']}")
print(f"Final State Route: {result_3['route']}")

 --- Running Test Case 1: What are your office hours next week? ---
--- Router Node: Analyzing Query: What are your office hours next week? ---


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1936: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


--- Router Decision: Route to 'general' ---
--- General Node: Processing Inquiry ---
Final State Response: Response from General Inquiry: Thank you for your general question about our services. We are looking into your request: 'What are your office hours next week?'.
Final State Route: general
 --- Running Test Case 2: I need to update the beneficiary on my term life insurance policy. ---
--- Router Node: Analyzing Query: I need to update the beneficiary on my term life insurance policy. ---


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1936: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


--- Router Decision: Route to 'policy' ---
--- Policy Node: Processing Inquiry ---
Final State Response: Response from Policy Management: Your query regarding your policy: 'I need to update the beneficiary on my term life insurance policy.' has been forwarded to a Policy Specialist. Expect a detailed response within 24 hours.
Final State Route: policy
 --- Running Test Case 3: What is your current outlook on the S&P 500 for the next quarter? ---
--- Router Node: Analyzing Query: What is your current outlook on the S&P 500 for the next quarter? ---


/usr/local/lib/python3.12/dist-packages/langchain_openai/chat_models/base.py:1936: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


--- Router Decision: Route to 'investment' ---
--- Investment Node: Processing Inquiry ---
Final State Response: Response from Investment Strategy: We are analyzing your investment-related query: 'What is your current outlook on the S&P 500 for the next quarter?'. Please note that investment advice is subject to market conditions.
Final State Route: investment


### Result Interpretation for Part A

The execution trace clearly demonstrates the power of **conditional routing** in LangGraph.

1.  The graph starts at the `router` node.
2.  The `router` node uses the LLM with a Pydantic schema to classify the query and determine the `route` (e.g., 'investment').
3.  The `add_conditional_edges` function then uses the `get_next_step` function to read the `route` from the state and dynamically transition to the correct agent node (`investment_agent`).
4.  The selected agent node executes and the graph terminates (`END`).

This pattern is crucial for building complex, multi-agent systems where different tasks require specialized logic or tools.

## Part B: Automate Compliance Report Generation Exercise

This exercise demonstrates a **static, sequential graph** used for a fixed workflow, such as automated document assembly. We will create a chain to generate a compliance report in three fixed steps: Data Collection, Risk Analysis, and Report Assembly.

In [13]:
# Define State and the three Sequential Node Functions
# 1. Define the State for the Compliance Report
class ReportState(TypedDict):
    report_title: str
    data_summary: str
    risk_analysis: str
    final_report: str

In [14]:

# 2. Define the Sequential Node Functions

def data_collection_node(state: ReportState) -> ReportState:
    """Simulates the first step: collecting and summarizing compliance data."""
    print("--- Step 1: Data Collection Node Executed ---")
    title = state["report_title"]

    # Simulated data collection result
    summary = f"Summary for {title}: The Q3 2025 transaction log shows 1,245,890 trades. 99.8% were executed within the 100ms latency target. Two minor breaches of the insider trading policy were flagged and resolved internally."

    return {"data_summary": summary}

def risk_analysis_node(state: ReportState) -> ReportState:
    """Uses the data summary to perform a risk analysis (LLM-powered)."""
    print("--- Step 2: Risk Analysis Node Executed ---")

    # Check if LLM is initialized before calling
    if llm is None:
        print("LLM not initialized. Skipping LLM call and using placeholder analysis.")
        analysis_result = "Placeholder Risk Analysis: Due to uninitialized LLM, a full analysis could not be performed. The system flagged two minor policy breaches which require follow-up."
        return {"risk_analysis": analysis_result}

    # LLM Prompt for Risk Analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a senior compliance officer. Based on the provided data summary, write a concise, professional risk analysis section for a compliance report. Focus on key risks and mitigation."),
        ("human", "Data Summary: {data_summary}")
    ])

    analysis_chain = analysis_prompt | llm

    # Invoke the LLM
    analysis_result = analysis_chain.invoke({"data_summary": state["data_summary"]}).content

    return {"risk_analysis": analysis_result}

def report_assembly_node(state: ReportState) -> ReportState:
    """Combines all sections into the final report document."""
    print("--- Step 3: Report Assembly Node Executed ---")

    final_report = f"""
# {state["report_title"]}

## 1. Executive Data Summary
{state["data_summary"]}

## 2. Risk Analysis
{state["risk_analysis"]}

---
*Report Generated by Automated Compliance Chain.*"""
    return {"final_report": final_report}

In [15]:
# Build and Compile the Static LangGraph Chain
# 1. Initialize the StateGraph
workflow_report = StateGraph(ReportState)

# 2. Add the nodes
workflow_report.add_node("data_collection", data_collection_node)
workflow_report.add_node("risk_analysis", risk_analysis_node)
workflow_report.add_node("report_assembly", report_assembly_node)

# 3. Set the entry point
workflow_report.set_entry_point("data_collection")

# 4. Add sequential edges (static chain)
workflow_report.add_edge("data_collection", "risk_analysis")
workflow_report.add_edge("risk_analysis", "report_assembly")

# 5. Add finish edge
workflow_report.add_edge("report_assembly", END)

# 6. Compile the graph
app_report = workflow_report.compile()

print("LangGraph for Compliance Report Generation compiled successfully.")

LangGraph for Compliance Report Generation compiled successfully.


In [16]:
# Run the Static Graph and print the final report
# Initial state with the report title
initial_state = {
    "report_title": "Q3 2025 Automated Compliance Report",
    "data_summary": "",
    "risk_analysis": "",
    "final_report": ""
}

print(f" --- Running Compliance Report Generation for: {initial_state['report_title']} ---")
final_result = app_report.invoke(initial_state)

print(" --- Generated Final Report ---")
print(final_result["final_report"])

 --- Running Compliance Report Generation for: Q3 2025 Automated Compliance Report ---
--- Step 1: Data Collection Node Executed ---
--- Step 2: Risk Analysis Node Executed ---
--- Step 3: Report Assembly Node Executed ---
 --- Generated Final Report ---

# Q3 2025 Automated Compliance Report

## 1. Executive Data Summary
Summary for Q3 2025 Automated Compliance Report: The Q3 2025 transaction log shows 1,245,890 trades. 99.8% were executed within the 100ms latency target. Two minor breaches of the insider trading policy were flagged and resolved internally.

## 2. Risk Analysis
Risk Analysis:

The Q3 2025 transaction data indicates a high level of operational efficiency, with 99.8% of trades executed within the established 100ms latency target, minimizing market risk related to execution delays. However, the identification of two minor insider trading policy breaches, although promptly resolved internally, highlights a residual compliance risk in information handling and employee cond

### Result Interpretation for Part B

This exercise demonstrates a **static, sequential workflow** where the execution path is fixed and predictable.

1.  The graph starts at `data_collection`.
2.  It proceeds directly to `risk_analysis` via a fixed edge.
3.  It then proceeds to `report_assembly` via another fixed edge.
4.  Each node updates the shared `ReportState` with its output, ensuring that the next node has the necessary context (e.g., `risk_analysis` uses `data_summary`).

This pattern is ideal for reliable, multi-step processes like data pipelines, document generation, or fixed business logic where the flow does not depend on an LLM decision.

## Conclusion

This exercise successfully demonstrated the two fundamental patterns of LangGraph:

| Pattern | LangGraph Feature | Use Case | Key Takeaway |
| :--- | :--- | :--- | :--- |
| **Part A: Client Query Routing** | Conditional Edges (`add_conditional_edges`) | Dynamic routing, multi-agent systems, tool selection. | LLMs can be used as intelligent routers to dynamically change the workflow path. |
| **Part B: Compliance Report** | Static Edges (`add_edge`) | Fixed, sequential processes, document assembly, data pipelines. | LangGraph provides a robust framework for managing state and flow in predictable, multi-step tasks. |

By mastering these two patterns, you are equipped to build sophisticated, production-ready LLM applications.